In [4]:
-- Tipos para representar estados
type Estado = (Int, Int)  -- (litros en jarra A, litros en jarra B)
type Nodo = Estado
type Costo = Int

-- Capacidades de las jarras
capA, capB :: Int
capA = 5
capB = 3


In [5]:
-- Verter de A hacia B (jarra de 5L a 3L)
transferirAB :: Estado -> Estado
transferirAB (a, b) =
  let espacioB = capB - b
      transferido = min a espacioB
  in (a - transferido, b + transferido)

-- Verter de B hacia A (jarra de 3L a 5L)
transferirBA :: Estado -> Estado
transferirBA (a, b) =
  let espacioA = capA - a
      transferido = min b espacioA
  in (a + transferido, b - transferido)


In [6]:
-- Estados vecinos: operaciones válidas
vecinos :: Estado -> [(Estado, Costo)]
vecinos (a, b) =
  [ ((capA, b), 1)           -- Llenar A
  , ((a, capB), 1)           -- Llenar B
  , ((0, b), 1)              -- Vaciar A
  , ((a, 0), 1)              -- Vaciar B
  , (transferirAB (a, b), 1) -- Verter A → B
  , (transferirBA (a, b), 1) -- Verter B → A
  ]


In [7]:
heuristica :: Estado -> Costo
heuristica (a, b) = min (abs (4 - a)) (abs (4 - b))


In [8]:
import Data.List (sortOn)

aStar :: (Nodo -> [(Nodo, Costo)])  -- función para obtener vecinos
      -> (Nodo -> Costo)             -- heurística
      -> Nodo                       -- estado inicial
      -> (Nodo -> Bool)             -- condición de meta
      -> [(Nodo, Costo)]            -- resultado: camino con costos

aStar sucesores heuristica inicio esMeta = buscar [(inicio, 0)] []
  where
    buscar [] _ = []
    buscar ((nodo, costo):cola) visitados
      | esMeta nodo = [(nodo, costo)]
      | nodo `elem` visitados = buscar cola visitados
      | otherwise =
          let nuevos = [ (v, costo + c) | (v, c) <- sucesores nodo, v `notElem` visitados ]
              ordenados = sortOn (\(v, c) -> c + heuristica v) (cola ++ nuevos)
          in (nodo, costo) : buscar ordenados (nodo : visitados)


In [9]:
inicio :: Estado
inicio = (0, 0)

esMeta :: Estado -> Bool
esMeta (a, b) = a == 4 || b == 4

solucion :: [(Estado, Costo)]
solucion = aStar vecinos heuristica inicio esMeta


In [10]:
main :: IO ()
main = do
  putStrLn "Secuencia de estados hasta alcanzar 4 litros:"
  mapM_ print solucion
